In [5]:
import vmas
import random

# Crear el entorno utilizando el escenario "navigation"
env = vmas.make_env(
    scenario="navigation",  # Cambiamos a "navigation"
    num_envs=1,             # Número de entornos paralelos
    device="cuda",          # Cambiar a "cpu" si no tienes GPU disponible
    continuous_actions=True,  # True para acciones continuas
    wrapper=None,            # Ningún wrapper por defecto
    max_steps=100,           # Define el horizonte, 100 pasos como ejemplo
    seed=42,                 # Seed para reproducibilidad
    dict_spaces=False,       # Espacio de observación y acción como tuplas
    grad_enabled=False,      # Gradientes deshabilitados
    terminated_truncated=False,  # Utiliza una sola bandera `done`
)

# Ciclo de interacción con el entorno
for _ in range(100):  # Número de episodios o iteraciones
    dict_actions = random.choice([True, False])  # Decidir si usar diccionario de acciones
    actions = {} if dict_actions else []  # Inicializar las acciones

    # Generar acciones aleatorias para cada agente
    for agent in env.agents:
        action = env.get_random_action(agent)
        if dict_actions:
            actions.update({agent.name: action})
        else:
            actions.append(action)
    
    # Paso en el entorno
    obs, rew, dones, info = env.step(actions)

    # Renderizado del entorno
    frame = env.render(
        mode="rgb_array",
        agent_index_focus=None,  # Enfocar la cámara en un agente (opcional)
        visualize_when_rgb=True,  # Mostrar visualización al obtener un array RGB
    )

    # Por si queremos guardar o procesar el frame
    # Ejemplo: guardar el frame como imagen usando OpenCV
    # import cv2
    # cv2.imwrite(f"frame_{_}.png", frame)




In [ ]:
# Limpiar recursos del entorno
env.viewer.close()

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import vmas

# Configuración del entorno
env = vmas.make_env(
    scenario="navigation",
    num_envs=1,
    device="cpu",  # Cambiar a "cuda" si queremos usar GPU
    continuous_actions=False,  # Ahora trabajamos con acciones discretas
    max_steps=200,
    seed=42,
)

# Hiperparámetros de PPO
GAMMA = 0.99          # Factor de descuento
LR = 1e-3             # Tasa de aprendizaje
EPS_CLIP = 0.2        # Rango de clipping en PPO
ENTROPY_COEF = 0.02   # Coeficiente para la entropía
VALUE_LOSS_COEF = 0.5 # Coeficiente para la pérdida del crítico
BATCH_SIZE = 64       # Tamaño del batch
EPOCHS = 4            # Épocas para actualizar la política

# Obtener lista de agentes en el entorno
agents = env.agents

# Número de agentes
num_agents = len(agents)

print(f"Number of agents in the environment: {num_agents}")


# Definición de la red Actor-Crítico
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.shared_layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
        )
        self.actor = nn.Linear(64, action_dim)  # Actor (logits para acciones discretas)
        self.critic = nn.Linear(64, 1)         # Crítico (valor del estado)

    def forward(self, x):
        shared = self.shared_layers(x)
        return self.actor(shared), self.critic(shared)

# Inicialización
# Reseteamos el entorno para obtener las primeras observaciones
initial_obs = env.reset()

# Determinar dimensiones de observaciones y acciones
obs_space = env.observation_space[0].shape[0]  # Primer agente
action_space = env.action_space[0].n           # Primer agente

# Imprimir las dimensiones para verificar
print(f"Observation space: {obs_space}, Action space: {action_space}")

policy = ActorCritic(obs_space, action_space).to("cpu")
optimizer = optim.Adam(policy.parameters(), lr=LR)

# Función de PPO
def compute_ppo_loss(old_log_probs, new_log_probs, advantages, values, returns):
    # Clipped Surrogate Objective
    ratio = torch.exp(new_log_probs - old_log_probs)
    clipped_ratio = torch.clamp(ratio, 1 - EPS_CLIP, 1 + EPS_CLIP)
    policy_loss = -torch.min(ratio * advantages, clipped_ratio * advantages).mean()

    # Value Function Loss
    value_loss = nn.MSELoss()(values, returns)

    # Entropy Loss (para explorar más)
    entropy_loss = -new_log_probs.mean()

    # Loss final
    return policy_loss + VALUE_LOSS_COEF * value_loss - ENTROPY_COEF * entropy_loss


def collect_trajectories(policy, env, gamma):
    states, actions, rewards, dones, log_probs, values = [], [], [], [], [], []
    state = env.reset()  # state es una lista o tupla con observaciones por agente
    total_reward = 0  # Acumular las recompensas totales
    step_rewards = []  # Recompensas individuales por paso

    for _ in range(env.max_steps):
        # Convertir las observaciones de todos los agentes a tensores
        state_tensor = torch.tensor(np.array(state), dtype=torch.float32)
        logits, value = policy(state_tensor)  # Predicción de la red

        # Selección de acciones discretas con probabilidad
        action_probs = torch.softmax(logits, dim=-1)
        action_distribution = torch.distributions.Categorical(action_probs)
        action = action_distribution.sample()
        log_prob = action_distribution.log_prob(action)
        
        # Interactuar con el entorno
        next_state, reward, done, _ = env.step(action.tolist())
        
        # Guardar las transiciones
        states.append(state_tensor)
        actions.append(action)
        rewards.append(torch.tensor(reward, dtype=torch.float32))  # Guardar recompensas por agente
        dones.append(done.clone().detach().to(dtype=torch.float32))  # Guardar banderas de finalización
        log_probs.append(log_prob)
        values.append(value)

        # Acumular recompensa y guardar por paso
        step_rewards.append(np.mean(reward))  # Promedio de recompensas por agente
        total_reward += np.mean(reward)

        if all(done):  # Si todos los agentes han terminado
            break
        state = next_state

    # Calcular retornos y ventajas
    returns = []
    G = torch.zeros_like(rewards[0])  # Inicializar con ceros del tamaño de la recompensa por agente
    for r, d in zip(reversed(rewards), reversed(dones)):
        G = r + gamma * G * (1 - d)  # Calcular retorno considerando las banderas `done`
        returns.insert(0, G)         # Insertar en el inicio para mantener el orden correcto

    returns = torch.cat(returns)  # Convertir lista de tensores a un tensor
    advantages = returns - torch.cat(values).detach()
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    # Imprimir recompensas paso a paso para depuración
    print(f"Step Rewards: {step_rewards}")

    # Retornar estados, acciones, retornos, log_probs, ventajas y recompensa total
    return states, actions, returns, log_probs, advantages, total_reward / len(step_rewards)



# Entrenamiento PPO
for epoch in range(1000):
    # Recolectar datos
    states, actions, returns, old_log_probs, advantages, total_reward = collect_trajectories(policy, env, GAMMA)

    # Convertir a tensores
    states = torch.stack(states)
    actions = torch.stack(actions)
    returns = returns.unsqueeze(1)
    old_log_probs = torch.stack(old_log_probs)
    advantages = advantages.unsqueeze(1)

    # Actualizar política y crítico
    for _ in range(EPOCHS):
        optimizer.zero_grad()  # Reiniciar gradientes antes de cada actualización
        logits, values = policy(states)
        action_probs = torch.softmax(logits, dim=-1)
        new_log_probs = torch.gather(torch.log(action_probs), dim=-1, index=actions.unsqueeze(-1)).squeeze(-1)

        # Calcular pérdida
        loss = compute_ppo_loss(
            old_log_probs.detach(),  # Desacoplar del grafo
            new_log_probs,
            advantages.detach(),  # Desacoplar ventajas para evitar acumulación de gradientes
            values,
            returns.detach()  # Desacoplar retornos
        )

        # Retropropagación
        loss.backward()
        optimizer.step()

    # Imprimir recompensa promedio por época
    print(f"Epoch {epoch}: Average Reward = {total_reward/env.max_steps:.2f}")





Number of agents in the environment: 4
Observation space: 18, Action space: 9
Step Rewards: [-0.0097459555, -0.01399377, -0.0022329092, -0.0105564, -0.023678005, -0.022732258, -0.032022506, -0.015824974, -0.004936695, 0.0052693486, 0.028895855, 0.03597212, 0.045841753, 0.03301543, 0.022581816, 0.023717165, 0.022160023, -0.00876537, -0.0059238374, -0.00486663, -0.018841296, -0.026874304, -0.021352679, -0.014106929, -0.021628201, -0.0040718317, 0.0048449337, 0.022208095, 0.017916083, 0.043294966, 0.035018504, 0.03555295, 0.022949576, 0.02510342, 0.015737593, -0.0065447986, -0.011897653, -0.024578363, -0.016358167, 0.007540822, 0.009846449, -0.010180056, -0.015475541, -0.00916025, -0.011279106, 0.008201152, 0.0059110224, -0.0019500852, 0.0060248375, -0.0019334555, -0.009509653, 0.021449268, 0.02759415, 0.027293652, 0.01602298, 0.011453897, 0.0065059215, -0.0054557174, -0.0060449988, -0.0038965344, 0.0014055073, -0.02894713, -0.027496606, -0.019444183, -0.030253023, -0.041158304, -0.057295

c:\Users\ilhui\anaconda3\envs\reto\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([800, 1])) that is different to the input size (torch.Size([200, 4, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 